# *OpenScope: Barcode*
<img src="https://qbi.uq.edu.au/files/55985/brain-barcode-renton-art-neuroscience-banner.jpg" width="380" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [2]:
import np_config
import np_jobs
import np_logging
import np_services
import np_session
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_barcode as Barcode

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [3]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [4]:
import contextlib
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [5]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select workflow
Re-run cell this cell if mouse ID is changed

In [6]:
np_workflows.mtrain_widget(mouse)

Select type of workflow to run

In [7]:
selected_workflow = Barcode.barcode_workflow_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [8]:
Barcode.validate_selected_workflow(selected_workflow.session, mouse)

experiment: Barcode.Ephys | Barcode.Hab = Barcode.new_experiment(mouse, user, selected_workflow.session)
session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, Barcode.Hab)

16:44 | web | INFO | Ephys(1271131032_366122_20230518) created


***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [9]:
np_workflows.check_hardware_widget()

In [10]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [10]:
experiment.initialize_and_test_services()

16:44 | np_services.utils | INFO | mousedirector is running on w10dtsm112722
16:44 | np_services.proxies | INFO | Proxy(MouseDirector) initialized: ready for use
16:45 | np_services.utils | INFO | sync_device is running on w10dtsm112719
16:45 | np_services.proxies | INFO | Proxy(Sync) initialized: ready for use
16:45 | np_services.proxies | INFO | VideoMVR initialized: ready for use
16:45 | np_services.utils | INFO | camviewer3d is running on w10dtsm112722
16:45 | np_services.proxies | INFO | Proxy(Cam3d) initialized: ready for use
16:45 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
16:45 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
16:45 | np_services.utils | INFO | camstim_agent is running on W10DT713938
16:45 | np_services.utils | INFO | phidget_server is running on W10DT713938
16:45 | np_services.proxies | ERROR | Error accessing SessionCamstim data path: \\W10DT713938\c$\ProgramData\AIBS_MPE\camstim\data
Traceback (most recent c

***
## Dip probes

In [ ]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_experiment_surface_image')

## Probe depths in dye

In [ ]:
if not hab:
    np_workflows.probe_depth_widget(session)

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()
    np_workflows.isi_widget(mouse.lims)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer & insertion notes

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

In [ ]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [ ]:
if not hab:
    np_workflows.print_countdown_timer(minutes=.1 if experiment.session_type.value == 'pretest' else 10)

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Start stimulus
mapping | main | opto 


Script and params determined by MTrain stage

In [ ]:
experiment.run_stim()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()

***
## Before removing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()

# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

In [ ]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)